In [ ]:
# Test for AI module: fdd_utils/ai_helper.py
from fdd_utils.ai_helper import AIHelper
 
# Define system prompt and user prompt
system_prompt = r"""You are a helpful assistant."""
user_prompt = r"""Whehre is HK?"""
 
# Choose the model type ('openai', 'local', 'deepseek')
model_type = "local"
 
# Create an instance of the AIHelper
ai_helper = AIHelper(model_type=model_type, user_prompt=user_prompt, system_prompt=system_prompt)
 
# Get the response from the AI model
response = ai_helper.get_response()
print(response)
 
# AI module test pass
 


In [ ]:
# Databook handling
from fdd_utils.process_databook import extract_data_from_excel
import warnings
 
# Ignore the user warning for data validation
warnings.simplefilter(action='ignore', category=UserWarning)
 
# Usage:
# Eng (Haining 3 entites): 221128.Project TK.Databook.JW.xlsx
# Semi Databook: deli.xlsx
databook_path = "inputs/221128.Project TK.Databook.JW.xlsx"
# Chi (single entity): 240624.联洋-databook.xlsx || 240627.东莞岭南-databook.xlsx
# databook_path = "inputs/240627.东莞岭南-databook.xlsx"
entity_name = 'Haining Wanpu' # Lianyang, Dongguan Lingnan
mode = "BS" # BS, IS, All
 
dfs, workbook_list, result_type, report_language = extract_data_from_excel(databook_path, entity_name, mode)
 
# workbook_list
print('Key list:', workbook_list)
 
# Example databook data for single item
# dfs['Cash']
 
# Excel processing module ok, individual steps inside workings/process_databook.ipynb
# Results -> Pass
# Remark: here key list = workbook tabname
# 联洋, 东莞岭南 IS, BS - pass
 
# one issue: 可能會有subtotal+細項的表達方法, 之後要小心，如果AI有bug要返呢度改
 



In [ ]:
dfs['Cash']
 


In [ ]:
report_language
 


In [ ]:
# Key reconciliation
 
# mapping.json
# "Cash", "AR", "Prepayments", "OR", "Inventory", "IP", "IA", "NCA", "NCL", "Other NCA", "Other CA", "AP", "Advances",
# "Tax payable", "OP", "Capital", "Reserve", "Capital reserve", "R/E", "OI", "OC", "Tax and Surcharges", "GA",
# "Fin Exp",  "Cr Loss", "Other Income", "Non-operating Income", "Non-operating Exp", "Income tax", "LT DTA", "Selling cost"
 
# pattern.json
# "Cash", "AR", "Prepayments", "OR", "IP", "NCA", "Other NCA", "Other CA", "AP", ## 少左: "Inventory"， "IA"， "NCL", "Advances"
# "Tax payable", "OP", "Capital", "Reserve", "OI", "OC", "Tax and Surcharges", "GA", ## 少左: "Capital reserve", "R/E"
# "Fin Exp", "Cr Loss", "Other Income", "Non-operating Income", "Non-operating Exp", "Income tax", "LT DTA" ## 少左: "Capital reserve", "R/E", "Selling cost"
 
# prompts.json
# "Cash", "AR", "Other NCA", "Other CA", "AP" ## 少左: "Prepayments", "OR", "Inventory", "IP", "IA", "NCA", "NCL", "Advances"
# "Reserve", "Capital reserve", "R/E", "OI", "OC", "Tax and Surcharges", "GA", ## 少左: "Tax payable", "OP", "Capital",
# "Fin Exp",  "Cr Loss", "Other Income", "Non-operating Income", "Non-operating Exp", "Income tax", "LT DTA", ## 少左: "Selling cost"
 
# 到時要加返上去 以mappings爲準
# Keys checked identical    
 


In [ ]:
# AI Agent 由呢度開始要+language
 
# Agent 1: 根據acct情況揀pattern+draft Input=language, key(account, pattern) Output=key+AI output
# Agent 2：對數字(+desc)係咪岩 Input=AI output + key(accounts)
# Agent 3: compliance -> 檢查内容係咪make sense
# Agent 4: translate
 
# 盡量+heuristic features 尤其on 容易睇到問題嘅位置
 
# Expand options: 1. industry 2.RAG
 
 


In [ ]:
import pandas as pd
from fdd_utils.content_generation import content_generate
 
agent = 'agent_1'
model_name = 'local'
report_language = 'Eng' # Eng, Chi
 
content_generate(agent, workbook_list, model_name, report_language, dfs)


In [ ]:
# Step 2: Run Agent 2 to verify and refine the output of Agent 1
for mapping_key in mapping_keys:
    result = load_and_verify(agent2, mapping_key)  # Implement a function to load and check results
 
    if result:
        # Implement AI processing for verification and refinement
        system_prompt = f"System prompt for {agent2} verification."
        user_prompt = f"User prompt containing data verification specifics for {mapping_key}."
           
        ai_helper = AIHelper(model_type=model_name, user_prompt=user_prompt, system_prompt=system_prompt)
        verification_response = ai_helper.get_response().strip()
 
        # Final verification step: ensure format compliance and log or output appropriately
        verification_response = verification_response.replace("\n\n", "\n").replace("\n \n", "\n")
        print(f"Verified Output for key {mapping_key}: {verification_response}")
 

In [ ]:
import os, yaml
 
def read_result_by_key(mapping_key, log_dir='fdd_utils'):
    intermediate_file = os.path.join(log_dir, 'intermediate_results.yml')
    if not os.path.exists(intermediate_file):
        print("Intermediate file does not exist.")
        return None
 
    with open(intermediate_file, 'r', encoding='utf-8') as file:
        data = yaml.safe_load(file) or {}
 
    result_data = data.get(mapping_key, None)
    return result_data.get('result') if result_data else None
 
# Example usage to read the result by key
result = read_result_by_key('AR')
print(result)
 
# ['Cash', 'AR', 'Prepayments', 'OR', 'Other current assets', 'Investment properties', 'Tax payable', 'OP', 'AP', 'Share capital']
# AR - Haining issue
# General: 1 d.p., 1000Kshould = M 3. all amount should be CNYxxxK or million
 
# Agent 1 completed with some issues

In [ ]:
##### Test may start here #####

In [ ]:
# Cell 1: Test New AIHelper with Agent Support
# Test for enhanced AI module: fdd_utils/ai_helper.py
from fdd_utils.ai_helper import AIHelper

# Test 1: Basic AIHelper with agent
print("="*60)
print("TEST 1: Basic AIHelper with Agent Support")
print("="*60)

# Create an instance with agent_1
ai_helper = AIHelper(
    model_type='local',  # or 'deepseek', 'openai'
    agent_name='agent_1',
    language='Eng',
    use_heuristic=False
)

# Simple test prompt
user_prompt = "Where is Hong Kong?"
system_prompt = "You are a helpful assistant."

# Get response
response = ai_helper.get_response(user_prompt, system_prompt)

print("\nResponse content:", response['content'])
print(f"Mode: {response['mode']}")
print(f"Duration: {response['duration']:.2f}s")
print(f"Tokens: {response['tokens_used']}")
print("\n✅ AIHelper test passed!")

In [ ]:
# Cell 2: Test Databook Extraction (Should Still Work)
# Test databook handling with language detection
from fdd_utils.process_databook import extract_data_from_excel
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)

print("="*60)
print("TEST 2: Databook Extraction")
print("="*60)

# Your existing paths
databook_path = "inputs/221128.Project TK.Databook.JW.xlsx"
entity_name = 'Haining Wanpu'
mode = "BS"  # BS, IS, All

# Extract data (same as before, but now with better language detection)
dfs, workbook_list, result_type, report_language = extract_data_from_excel(
    databook_path, entity_name, mode
)

print(f"\n✅ Extracted {len(workbook_list)} worksheets")
print(f"Result type: {result_type}")
print(f"Detected language: {report_language}")
print(f"Keys: {workbook_list[:5]}...")  # Show first 5

# Check a sample
if 'Cash' in dfs:
    print(f"\nSample data (Cash):")
    print(dfs['Cash'].head())

In [ ]:
# Cell 3: Test New AI Pipeline - Quick Method
# Test the new AI pipeline - Quick method
from fdd_utils.ai_pipeline import run_quick_pipeline

print("="*60)
print("TEST 3: Quick Pipeline Execution")
print("="*60)

# Quick test with heuristic mode (no API calls, fast)
print("\nRunning with HEURISTIC mode (no AI calls)...")
results = run_quick_pipeline(
    databook_path=databook_path,
    entity_name=entity_name,
    mode='BS',
    model_type='local',
    language=report_language or 'Eng'
)

print(f"\n✅ Quick pipeline completed!")
print(f"Processed {len(results)} items")

# Show first result
first_key = list(results.keys())[0] if results else None
if first_key:
    print(f"\nSample result for '{first_key}':")
    print(results[first_key][:200] + "..." if len(results[first_key]) > 200 else results[first_key])

In [ ]:
# Cell 4: Test Full Pipeline with Orchestrator
# Test with the orchestrator (recommended method)
from fdd_utils.ai_pipeline import AIPipelineOrchestrator

print("="*60)
print("TEST 4: Full AI Pipeline with Orchestrator")
print("="*60)

# Initialize orchestrator
orchestrator = AIPipelineOrchestrator(config_path='fdd_utils/config.yml')

# Load data (reusing the variables from Cell 2)
print("\nLoading data...")
data_info = orchestrator.load_data(
    databook_path=databook_path,
    entity_name=entity_name,
    mode='BS'
)

print(f"✅ Loaded {len(data_info['mapping_keys'])} worksheets")
print(f"Detected language: {data_info['report_language']}")

# Run full pipeline with HEURISTIC mode first (for testing, no API costs)
print("\n" + "="*60)
print("Running FULL PIPELINE (Heuristic Mode - Testing)")
print("="*60)

results = orchestrator.run_full_pipeline(
    model_type='local',
    language=data_info['report_language'],
    use_heuristic=True,  # Set to False to use real AI
    use_multithreading=True,
    max_workers=4
)

# Print summary
orchestrator.print_results_summary()

# Save results
orchestrator.save_results('fdd_utils/output/test_results.yml')
print("\n✅ Full pipeline test completed!")

In [ ]:
# Cell 5: Test Individual Agents
# Test running individual agents
print("="*60)
print("TEST 5: Individual Agents")
print("="*60)

# Reinitialize orchestrator with fresh data
orchestrator = AIPipelineOrchestrator()
orchestrator.load_data(databook_path, entity_name, 'BS')

# Run only Agent 1 (Content Generator)
print("\n--- Running Agent 1 Only ---")
results_agent_1 = orchestrator.run_agent_only(
    agent_name='agent_1',
    model_type='local',
    language='Eng',
    use_heuristic=True  # Set to False for real AI
)

print(f"✅ Agent 1 completed: {len(results_agent_1)} items")

# Show a sample result
sample_key = list(results_agent_1.keys())[0]
print(f"\nAgent 1 output for '{sample_key}':")
print(results_agent_1[sample_key][:300] + "...")

# Run Agent 2 with Agent 1's results
print("\n--- Running Agent 2 Only (with Agent 1 results) ---")
results_agent_2 = orchestrator.run_agent_only(
    agent_name='agent_2',
    previous_results=results_agent_1,
    model_type='local',
    language='Eng',
    use_heuristic=True
)

print(f"✅ Agent 2 completed: {len(results_agent_2)} items")

In [ ]:
# Cell 6: Test Sequential Agents
# Test running specific agents in sequence
print("="*60)
print("TEST 6: Sequential Agents (Custom Order)")
print("="*60)

orchestrator = AIPipelineOrchestrator()
orchestrator.load_data(databook_path, entity_name, 'BS')

# Run only agents 1, 2, and 4 (skip agent 3 if desired)
results = orchestrator.run_sequential_agents(
    agents=['agent_1', 'agent_2', 'agent_4'],  # Custom agent sequence
    model_type='local',
    language='Eng',
    use_heuristic=True  # Set to False for real AI
)

print(f"\n✅ Sequential agents completed: {len(results)} items")

# Compare with full pipeline
orchestrator.print_results_summary()

In [ ]:
# Cell 7: Test with REAL AI (DeepSeek)
# Test with real AI (replace heuristic with actual AI)
print("="*60)
print("TEST 7: Real AI Pipeline (DeepSeek)")
print("="*60)
print("⚠️  This will use API credits!")

# Make sure your API key is configured in config.yml first!
confirm = input("Run with real AI? This will use API credits. (yes/no): ")

if confirm.lower() == 'yes':
    orchestrator = AIPipelineOrchestrator()
    orchestrator.load_data(databook_path, entity_name, 'BS')
    
    # Run with REAL AI
    results = orchestrator.run_full_pipeline(
        model_type='local',  # or 'deepseek' if configured
        language=report_language or 'Eng',
        use_heuristic=False,  # Real AI!
        use_multithreading=True,
        max_workers=4
    )
    
    # Save results
    orchestrator.save_results('fdd_utils/output/real_ai_results.yml')
    orchestrator.print_results_summary()
    
    # Show a detailed result
    sample_key = 'Cash' if 'Cash' in results else list(results.keys())[0]
    print(f"\n{'='*60}")
    print(f"Detailed result for '{sample_key}':")
    print(f"{'='*60}")
    print(results[sample_key])
    
    print("\n✅ Real AI pipeline completed!")
else:
    print("Skipped real AI test.")

In [ ]:
# Cell 8: View Results and Logs
# View results and check logs
import yaml
import os
from glob import glob

print("="*60)
print("TEST 8: View Results and Logs")
print("="*60)

# Check intermediate results (if exists from old system)
intermediate_file = 'fdd_utils/intermediate_results.yml'
if os.path.exists(intermediate_file):
    with open(intermediate_file, 'r', encoding='utf-8') as f:
        old_results = yaml.safe_load(f)
    print(f"\n📁 Old intermediate results: {len(old_results)} items")

# Check new log files
log_dir = 'fdd_utils/logs'
if os.path.exists(log_dir):
    log_files = sorted(glob(os.path.join(log_dir, 'ai_processing_*.log')))
    data_files = sorted(glob(os.path.join(log_dir, 'ai_data_*.yml')))
    
    print(f"\n📋 Log files: {len(log_files)}")
    print(f"📊 Data files: {len(data_files)}")
    
    # Show latest log file name
    if log_files:
        latest_log = log_files[-1]
        print(f"\nLatest log: {os.path.basename(latest_log)}")
        
        # Read last 20 lines
        with open(latest_log, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            print("\nLast 20 lines of log:")
            print("".join(lines[-20:]))
    
    # Show latest data file summary
    if data_files:
        latest_data = data_files[-1]
        print(f"\n📊 Latest data file: {os.path.basename(latest_data)}")
        
        with open(latest_data, 'r', encoding='utf-8') as f:
            data = yaml.safe_load(f)
            
        if 'summary' in data:
            print("\nSummary:")
            for key, value in data['summary'].items():
                print(f"  {key}: {value}")

# Check saved results
output_file = 'fdd_utils/output/test_results.yml'
if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        results = yaml.safe_load(f)
    print(f"\n✅ Saved results file exists: {len(results)} items")

# ========================================
# COMPREHENSIVE TESTING SECTION
# ========================================
# The cells below provide comprehensive testing of the improved AI pipeline
# with fixed agent prompts (agents 2, 3, 4 now output only content, no meta-commentary)


In [ ]:
# Cell 1: Test Data Loading and Language Detection
print("="*70)
print("TEST 1: Data Loading and Language Detection")
print("="*70)

from fdd_utils.process_databook import extract_data_from_excel
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)

# Test with English databook
databook_path = "inputs/221128.Project TK.Databook.JW.xlsx"
entity_name = 'Haining Wanpu'
mode = "BS"  # BS, IS, All

# Extract data
dfs, workbook_list, result_type, report_language = extract_data_from_excel(
    databook_path, entity_name, mode
)

print(f"\n✅ Data Loading Results:")
print(f"   - Extracted worksheets: {len(workbook_list)}")
print(f"   - Result type: {result_type}")
print(f"   - Detected language: {report_language}")
print(f"   - Worksheet keys: {workbook_list}")

# Display sample data
if 'Cash' in dfs:
    print(f"\n📊 Sample data for 'Cash':")
    print(dfs['Cash'])
    
print("\n" + "="*70 + "\n")


In [ ]:
# Cell 2: Test Full AI Pipeline with HEURISTIC Mode (Fast Testing)
print("="*70)
print("TEST 2: Full AI Pipeline - HEURISTIC Mode (No API Calls)")
print("="*70)

from fdd_utils.content_generation import run_ai_pipeline, save_results
import os

# Run pipeline in HEURISTIC mode (no actual AI calls, instant testing)
print("\n🔄 Running 4-agent pipeline in HEURISTIC mode...")
print("   This tests the pipeline structure without making API calls\n")

results_heuristic = run_ai_pipeline(
    mapping_keys=workbook_list[:3],  # Test with first 3 items
    dfs=dfs,
    model_type='local',
    language=report_language,
    use_heuristic=True,  # HEURISTIC mode = no API calls
    use_multithreading=True,
    max_workers=4
)

print(f"\n✅ HEURISTIC Pipeline Results:")
print(f"   - Items processed: {len(results_heuristic)}")
print(f"   - Agents per item: {list(results_heuristic[workbook_list[0]].keys()) if results_heuristic else 'None'}")

# Check structure
if results_heuristic and workbook_list[0] in results_heuristic:
    sample_key = workbook_list[0]
    print(f"\n📋 Sample result structure for '{sample_key}':")
    for agent, content in results_heuristic[sample_key].items():
        print(f"   - {agent}: {len(content)} chars")
        if agent == 'final':
            print(f"      Preview: {content[:100]}...")

print("\n" + "="*70 + "\n")


In [ ]:
# Cell 3: Test with REAL AI (Local Model - Single Item)
print("="*70)
print("TEST 3: Real AI Pipeline - Single Item Test")
print("="*70)
print("⚠️  This will make actual AI API calls to local model")
print("")

# Confirm before running
import sys
confirm = input("Run with REAL AI? This will use local AI model. (yes/no): ")

if confirm.lower() == 'yes':
    print("\n🔄 Running pipeline with REAL AI (Local Model)...\n")
    
    # Test with just ONE item first
    test_keys = [workbook_list[0]]  # Test with 'Cash' only
    test_dfs = {k: dfs[k] for k in test_keys}
    
    results_real_ai = run_ai_pipeline(
        mapping_keys=test_keys,
        dfs=test_dfs,
        model_type='local',  # Using local model
        language=report_language,
        use_heuristic=False,  # REAL AI!
        use_multithreading=False,  # Single-threaded for testing
        max_workers=1
    )
    
    print(f"\n✅ Real AI Pipeline Results:")
    print(f"   - Items processed: {len(results_real_ai)}")
    
    # Display detailed results
    if results_real_ai and test_keys[0] in results_real_ai:
        sample_key = test_keys[0]
        print(f"\n📄 Detailed results for '{sample_key}':")
        print("="*70)
        
        for agent_name in ['agent_1', 'agent_2', 'agent_3', 'agent_4', 'final']:
            if agent_name in results_real_ai[sample_key]:
                content = results_real_ai[sample_key][agent_name]
                print(f"\n{agent_name.upper()}:")
                print(f"Length: {len(content)} chars")
                print(f"Content: {content}")
                print("-"*70)
        
        # Check for meta-commentary (should be removed by clean_agent_output)
        final_content = results_real_ai[sample_key].get('final', '')
        meta_keywords = ['verified', 'corrected', 'refined', 'formatted', 'output:', 
                        '已验证', '已更正', '经过']
        found_meta = [kw for kw in meta_keywords if kw.lower() in final_content.lower()]
        
        if found_meta:
            print(f"\n⚠️  WARNING: Found potential meta-commentary: {found_meta}")
        else:
            print(f"\n✅ No meta-commentary detected in final output!")
    
    print("\n" + "="*70 + "\n")
else:
    print("\n⏭️  Skipped real AI test.\n")


In [ ]:
# Cell 4: Test Full Pipeline with ALL Items (Real AI)
print("="*70)
print("TEST 4: Full Pipeline with ALL Items - Real AI")
print("="*70)
print("⚠️  This will process ALL items with real AI - will take time!")
print(f"    Total items to process: {len(workbook_list)}")
print("")

confirm = input("Run FULL pipeline with REAL AI? (yes/no): ")

if confirm.lower() == 'yes':
    print("\n🔄 Running FULL pipeline with REAL AI...")
    print("   Using multi-threading for faster processing\n")
    
    results_full = run_ai_pipeline(
        mapping_keys=workbook_list,
        dfs=dfs,
        model_type='local',
        language=report_language,
        use_heuristic=False,  # REAL AI
        use_multithreading=True,  # Multi-threading enabled
        max_workers=4
    )
    
    print(f"\n✅ Full Pipeline Completed!")
    print(f"   - Total items processed: {len(results_full)}")
    print(f"   - Successful: {sum(1 for v in results_full.values() if 'final' in v)}")
    print(f"   - Failed: {sum(1 for v in results_full.values() if 'final' not in v)}")
    
    # Save results
    output_path = 'fdd_utils/output/results.yml'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    save_results(results_full, output_path)
    
    print(f"\n💾 Results saved to: {output_path}")
    
    # Display summary of first 3 items
    print(f"\n📋 Summary of First 3 Items:")
    print("="*70)
    for i, key in enumerate(list(results_full.keys())[:3], 1):
        if 'final' in results_full[key]:
            final = results_full[key]['final']
            print(f"\n{i}. {key}:")
            print(f"   Length: {len(final)} chars")
            print(f"   Preview: {final[:150]}...")
    
    print("\n" + "="*70 + "\n")
else:
    print("\n⏭️  Skipped full pipeline test.\n")


In [ ]:
# Cell 5: Analyze Results and Check for Issues
print("="*70)
print("TEST 5: Analyze Results and Validate Output Quality")
print("="*70)

import yaml
import os

# Check if results.yml exists
results_path = 'fdd_utils/output/results.yml'

if os.path.exists(results_path):
    print(f"\n📊 Loading results from: {results_path}\n")
    
    with open(results_path, 'r', encoding='utf-8') as f:
        results = yaml.safe_load(f)
    
    print(f"✅ Loaded {len(results)} items from results.yml\n")
    
    # Analyze each item
    print("="*70)
    print("QUALITY ANALYSIS")
    print("="*70)
    
    issues_found = []
    
    for key, agents_output in results.items():
        print(f"\n📌 {key}:")
        
        # Check if all agents completed
        expected_agents = ['agent_1', 'agent_2', 'agent_3', 'agent_4', 'final']
        missing_agents = [a for a in expected_agents if a not in agents_output]
        
        if missing_agents:
            print(f"   ⚠️  Missing agents: {missing_agents}")
            issues_found.append(f"{key}: Missing agents {missing_agents}")
        else:
            print(f"   ✅ All agents completed")
        
        # Check agent_2 and agent_3 for meta-commentary
        for agent_name in ['agent_2', 'agent_3', 'agent_4', 'final']:
            if agent_name in agents_output:
                content = agents_output[agent_name]
                
                # Check for meta-commentary keywords
                meta_patterns = [
                    'verified output:', 'corrected output:', 'refined output:',
                    'after verification', 'after refining', 'corrections made:',
                    'i verified', 'i corrected', 'i refined',
                    '已验证输出', '已更正输出', '经过验证', '经过精炼', '所做更正'
                ]
                
                found_meta = [p for p in meta_patterns if p in content.lower()]
                
                if found_meta:
                    print(f"   ⚠️  {agent_name}: Found meta-commentary: {found_meta}")
                    issues_found.append(f"{key} - {agent_name}: Meta-commentary found")
                else:
                    print(f"   ✅ {agent_name}: Clean output (no meta-commentary)")
        
        # Check final output length
        if 'final' in agents_output:
            final_len = len(agents_output['final'])
            print(f"   📏 Final output length: {final_len} chars")
            
            if final_len < 50:
                print(f"   ⚠️  Warning: Final output seems too short")
                issues_found.append(f"{key}: Final output too short ({final_len} chars)")
    
    # Summary
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    
    if issues_found:
        print(f"\n⚠️  Found {len(issues_found)} issues:")
        for issue in issues_found:
            print(f"   - {issue}")
    else:
        print("\n✅ NO ISSUES FOUND! All outputs are clean and complete.")
    
    print("\n" + "="*70 + "\n")
else:
    print(f"\n❌ Results file not found: {results_path}")
    print("   Please run Test 4 first to generate results.\n")


In [ ]:
# Cell 6: View Logs and Detailed Agent Outputs
print("="*70)
print("TEST 6: View Logs and Agent Processing Details")
print("="*70)

import glob
from datetime import datetime

# Find latest log files
log_dir = 'fdd_utils/logs'

if os.path.exists(log_dir):
    log_files = sorted(glob.glob(os.path.join(log_dir, 'ai_processing_*.log')))
    data_files = sorted(glob.glob(os.path.join(log_dir, 'ai_data_*.yml')))
    
    print(f"\n📋 Found {len(log_files)} log files")
    print(f"📊 Found {len(data_files)} data files\n")
    
    # Show latest log file
    if log_files:
        latest_log = log_files[-1]
        print(f"📄 Latest log file: {os.path.basename(latest_log)}")
        
        # Read last 30 lines
        with open(latest_log, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            print("\nLast 30 lines:")
            print("="*70)
            print("".join(lines[-30:]))
    
    # Show latest data file summary
    if data_files:
        latest_data = data_files[-1]
        print(f"\n📊 Latest data file: {os.path.basename(latest_data)}")
        
        with open(latest_data, 'r', encoding='utf-8') as f:
            data = yaml.safe_load(f)
        
        if 'summary' in data:
            print("\n📈 Pipeline Summary:")
            print("="*70)
            for key, value in data['summary'].items():
                print(f"   {key}: {value}")
        
        # Show agent outputs for first item
        if 'processing_results' in data:
            first_key = list(data['processing_results'].keys())[0]
            print(f"\n📝 Detailed Output for First Item: '{first_key}'")
            print("="*70)
            
            for agent_name, agent_data in data['processing_results'][first_key].items():
                if 'output' in agent_data:
                    print(f"\n{agent_name.upper()}:")
                    print(f"   Duration: {agent_data.get('duration', 0):.2f}s")
                    print(f"   Tokens: {agent_data.get('tokens_used', 0)}")
                    print(f"   Mode: {agent_data.get('mode', 'unknown')}")
                    print(f"   Output length: {agent_data.get('content_length', 0)} chars")
                    print(f"   Output preview: {agent_data['output'][:150]}...")
                    print("-"*70)
else:
    print(f"\n❌ Log directory not found: {log_dir}")

print("\n" + "="*70 + "\n")


In [ ]:
# Cell 7: Compare Agent Outputs (Show Evolution)
print("="*70)
print("TEST 7: Compare Agent Outputs - Content Evolution")
print("="*70)

results_path = 'fdd_utils/output/results.yml'

if os.path.exists(results_path):
    with open(results_path, 'r', encoding='utf-8') as f:
        results = yaml.safe_load(f)
    
    # Pick first item with all agents
    sample_key = None
    for key, agents in results.items():
        if all(agent in agents for agent in ['agent_1', 'agent_2', 'agent_3', 'agent_4', 'final']):
            sample_key = key
            break
    
    if sample_key:
        print(f"\n📊 Content Evolution for: '{sample_key}'")
        print("="*70)
        
        agents_data = results[sample_key]
        
        for agent_name in ['agent_1', 'agent_2', 'agent_3', 'agent_4', 'final']:
            if agent_name in agents_data:
                content = agents_data[agent_name]
                print(f"\n{agent_name.upper()}:")
                print(f"   Length: {len(content)} chars")
                print(f"   Content:\n   {content}")
                print("-"*70)
        
        # Show length changes
        print("\n📏 Length Evolution:")
        lengths = {agent: len(agents_data[agent]) for agent in ['agent_1', 'agent_2', 'agent_3', 'agent_4', 'final'] if agent in agents_data}
        for agent, length in lengths.items():
            print(f"   {agent}: {length} chars")
        
    else:
        print("\n⚠️  No item found with all agents completed")
else:
    print(f"\n❌ Results file not found: {results_path}")

print("\n" + "="*70 + "\n")


In [ ]:
# Cell 8: Extract Final Contents for Use
print("="*70)
print("TEST 8: Extract Final Contents for Further Processing")
print("="*70)

from fdd_utils.content_generation import extract_final_contents

results_path = 'fdd_utils/output/results.yml'

if os.path.exists(results_path):
    with open(results_path, 'r', encoding='utf-8') as f:
        results = yaml.safe_load(f)
    
    # Extract final contents only
    final_contents = extract_final_contents(results)
    
    print(f"\n✅ Extracted {len(final_contents)} final contents")
    print("\n📄 Final Contents:")
    print("="*70)
    
    for key, content in final_contents.items():
        print(f"\n{key}:")
        print(f"   {content}")
        print("-"*70)
    
    # Save final contents separately
    final_output_path = 'fdd_utils/output/final_contents.yml'
    with open(final_output_path, 'w', encoding='utf-8') as f:
        yaml.dump(final_contents, f, default_flow_style=False, allow_unicode=True)
    
    print(f"\n💾 Final contents saved to: {final_output_path}")
    print("\n✅ These final contents can now be used for PowerPoint generation!")
    
else:
    print(f"\n❌ Results file not found: {results_path}")

print("\n" + "="*70 + "\n")


## 🎯 TESTING COMPLETE!

### Summary of Tests:
1. ✅ **Test 1**: Data loading and language detection
2. ✅ **Test 2**: Full AI pipeline (HEURISTIC mode - fast testing)
3. ✅ **Test 3**: Real AI pipeline (Single item - with local model)
4. ✅ **Test 4**: Full pipeline with ALL items (Real AI)
5. ✅ **Test 5**: Analyze results and validate output quality
6. ✅ **Test 6**: View logs and detailed agent outputs
7. ✅ **Test 7**: Compare agent outputs (content evolution)
8. ✅ **Test 8**: Extract final contents for further processing

### Key Improvements Made:
- ✅ Fixed Agent 2 & 3 prompts to output ONLY revised content (no meta-commentary)
- ✅ Clarified Agent 4 purpose and output format
- ✅ Added `clean_agent_output()` function to remove any remaining meta-commentary
- ✅ Improved error handling and logging
- ✅ Added comprehensive testing suite

### What is agent_4 and 'final' for?
- **agent_4**: Format Checker - performs final formatting checks (currency format, quotation marks, numbering format, language consistency)
- **final**: Stores the agent_4 output as the final result ready for use in PowerPoint generation

### Files Generated:
- `fdd_utils/output/results.yml` - Full results with all agent outputs
- `fdd_utils/output/final_contents.yml` - Only final contents for PowerPoint
- `fdd_utils/logs/ai_processing_*.log` - Processing logs
- `fdd_utils/logs/ai_data_*.yml` - Detailed processing data with prompts and outputs


In [ ]:
# Cell 9: Performance Comparison
# Compare performance: single-threaded vs multi-threaded
import time

print("="*60)
print("TEST 9: Performance Comparison")
print("="*60)

# Test with smaller dataset or heuristic mode for speed
orchestrator = AIPipelineOrchestrator()
orchestrator.load_data(databook_path, entity_name, 'BS')

# Take only first 5 items for testing
test_keys = workbook_list[:5]
test_dfs = {k: dfs[k] for k in test_keys if k in dfs}

print(f"\nTesting with {len(test_keys)} items")

# Test 1: Without multi-threading
print("\n--- Test 1: Single-threaded ---")
from fdd_utils.content_generation import ai_pipeline_full

start = time.time()
results_single = ai_pipeline_full(
    mapping_keys=test_keys,
    dfs=test_dfs,
    model_type='local',
    language='Eng',
    use_heuristic=True,
    use_multithreading=False
)
time_single = time.time() - start

print(f"Time: {time_single:.2f}s")

# Test 2: With multi-threading
print("\n--- Test 2: Multi-threaded (4 workers) ---")
start = time.time()
results_multi = ai_pipeline_full(
    mapping_keys=test_keys,
    dfs=test_dfs,
    model_type='local',
    language='Eng',
    use_heuristic=True,
    use_multithreading=True,
    max_workers=4
)
time_multi = time.time() - start

print(f"Time: {time_multi:.2f}s")

# Compare
if time_multi > 0:
    speedup = time_single / time_multi
    print(f"\n{'='*60}")
    print(f"⚡ Speedup: {speedup:.2f}x faster with multi-threading")
    print(f"{'='*60}")